In [1]:
import requests
from bs4 import BeautifulSoup

In [19]:
base_url = r'https://www.sec.gov'

# 10_K file name
# file_name_10_K = 'https://www.sec.gov/Archives/edgar/data/1102934/0001102934-21-000052.txt'
file_name_10_k = 'https://www.sec.gov/Archives/edgar/data/897723/0000897723-21-000028.txt'
# documents_url = r'https://www.sec.gov/Archives/edgar/data/1102934/000110293421000052/index.json'
documents_url = file_name_10_K.replace('-', '').replace('.txt', '/index.json')

headers = {'User-Agent': 'miaowa.pro'}

content = requests.get(documents_url, headers=headers).json()

for file in content['directory']['item']:
    
    if file['name'] == 'FilingSummary.xml':
        
        xml_summary = base_url + content['directory']['name'] + '/' + file['name']
        
        print('-' * 100)
        print('File name:', file['name'])
        print('File paht:', xml_summary)

----------------------------------------------------------------------------------------------------
File name: FilingSummary.xml
File paht: https://www.sec.gov/Archives/edgar/data/1102934/000110293421000052/FilingSummary.xml


```xml
<MyReports>
  <Report instance="ccmp-20210930.htm">
    <IsDefault>false</IsDefault>
    <HasEmbeddedReports>false</HasEmbeddedReports>
    <HtmlFileName>R3.htm</HtmlFileName>
    <LongName>100020003 - Statement - CONSOLIDATED STATEMENTS OF COMPREHENSIVE (LOSS) INCOME</LongName>
    <ReportType>Sheet</ReportType>
    <Role>http://cabotcmp.com/role/CONSOLIDATEDSTATEMENTSOFCOMPREHENSIVELOSSINCOME</Role>
    <ShortName>CONSOLIDATED STATEMENTS OF COMPREHENSIVE (LOSS) INCOME</ShortName>
    <MenuCategory>Statements</MenuCategory>
    <Position>3</Position>
  </Report>
</MyReports>
```

In [20]:
base_url = xml_summary.replace('FilingSummary.xml', '')

content = requests.get(xml_summary, headers=headers).content
soup = BeautifulSoup(content, 'lxml')

reports = soup.find('myreports')
# print(reports)

master_reports = []

for report in reports.find_all('report')[:-1]:
    
    report_dict = {}
    report_dict['SHORT_NAME'] = report.shortname.text
    report_dict['LONG_NAME'] = report.longname.text
    report_dict['POSITION'] = report.position.text
    report_dict['CATEGORY'] = report.menucategory.text
    report_dict['URL'] = base_url + report.htmlfilename.text
    
    if report_dict['CATEGORY'] == 'Statements':
        master_reports.append(report_dict)
    
master_reports
    

[{'SHORT_NAME': 'CONSOLIDATED STATEMENTS OF (LOSS) INCOME',
  'LONG_NAME': '100010002 - Statement - CONSOLIDATED STATEMENTS OF (LOSS) INCOME',
  'POSITION': '2',
  'CATEGORY': 'Statements',
  'URL': 'https://www.sec.gov/Archives/edgar/data/1102934/000110293421000052/R2.htm'},
 {'SHORT_NAME': 'CONSOLIDATED STATEMENTS OF COMPREHENSIVE (LOSS) INCOME',
  'LONG_NAME': '100020003 - Statement - CONSOLIDATED STATEMENTS OF COMPREHENSIVE (LOSS) INCOME',
  'POSITION': '3',
  'CATEGORY': 'Statements',
  'URL': 'https://www.sec.gov/Archives/edgar/data/1102934/000110293421000052/R3.htm'},
 {'SHORT_NAME': 'CONSOLIDATED BALANCE SHEETS',
  'LONG_NAME': '100030004 - Statement - CONSOLIDATED BALANCE SHEETS',
  'POSITION': '4',
  'CATEGORY': 'Statements',
  'URL': 'https://www.sec.gov/Archives/edgar/data/1102934/000110293421000052/R4.htm'},
 {'SHORT_NAME': 'CONSOLIDATED BALANCE SHEETS (Parenthetical)',
  'LONG_NAME': '100040005 - Statement - CONSOLIDATED BALANCE SHEETS (Parenthetical)',
  'POSITION': '5',

In [27]:
statements_url = []

for report_dict in master_reports:
    
    # define the statements we want to look for.
    item1 = r"CONSOLIDATED BALANCE SHEETS"
    item2 = r"CONSOLIDATED BALANCE SHEETS (PARENTHETICAL)"
    item3 = r"CONSOLIDATED STATEMENTS OF (LOSS) INCOME"
    item4 = r"CONSOLIDATED STATEMENTS OF COMPREHENSIVE (LOSS) INCOME"
    item5 = r"CONSOLIDATED STATEMENTS OF CASH FLOWS"
    
    # store them in a list.
    report_list = [item1, item2, item3, item4, item5]
    
#     print(report_dict['SHORT_NAME'])
    if report_dict['SHORT_NAME'].upper() in report_list:
        
        print('-' * 100)
#         print(report_dict['POSITION'])
        print(report_dict['SHORT_NAME'])
        print(report_dict['URL'])
        statements_url.append(report_dict['URL'])
        


----------------------------------------------------------------------------------------------------
CONSOLIDATED STATEMENTS OF (LOSS) INCOME
https://www.sec.gov/Archives/edgar/data/1102934/000110293421000052/R2.htm
----------------------------------------------------------------------------------------------------
CONSOLIDATED STATEMENTS OF COMPREHENSIVE (LOSS) INCOME
https://www.sec.gov/Archives/edgar/data/1102934/000110293421000052/R3.htm
----------------------------------------------------------------------------------------------------
CONSOLIDATED BALANCE SHEETS
https://www.sec.gov/Archives/edgar/data/1102934/000110293421000052/R4.htm
----------------------------------------------------------------------------------------------------
CONSOLIDATED BALANCE SHEETS (Parenthetical)
https://www.sec.gov/Archives/edgar/data/1102934/000110293421000052/R5.htm
----------------------------------------------------------------------------------------------------
CONSOLIDATED STATEMENTS OF CASH